## California Housing Prices 

* Data link https://www.kaggle.com/datasets/camnugent/california-housing-prices 

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = 'data/housing.csv'
df = pd.read_csv(data)

In [3]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [4]:
df.shape

(20640, 10)

## Data preparation

* Select only the features from above and fill in the missing values with 0.
* Create a new column rooms_per_household by dividing the column total_rooms by the column households from dataframe.
* Create a new column bedrooms_per_room by dividing the column total_bedrooms by the column total_rooms from dataframe.
* Create a new column population_per_household by dividing the column population by the column households from dataframe.

In [5]:
features = ['latitude', 'longitude', 'housing_median_age',
            'total_rooms', 'total_bedrooms', 'population',
            'households', 'median_income', 'median_house_value',
            'ocean_proximity']

In [6]:
df = df[features]

In [7]:
df.isnull().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

Only column 'total_bedrooms' contains missing values. Fill missing values by 0

In [8]:
df = df.fillna(0)

In [12]:
df['rooms_per_household'] = df.total_rooms / df.households

In [13]:
df['bedrooms_per_room'] = df.total_bedrooms / df.total_rooms

In [14]:
df['population_per_household'] = df.population / df.households

In [15]:
df.head()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,bedrooms_per_room,population_per_household,rooms_per_household
0,37.88,-122.23,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,0.146591,2.555556,6.984127
1,37.86,-122.22,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,0.155797,2.109842,6.238137
2,37.85,-122.24,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,0.129516,2.802260,8.288136
3,37.85,-122.25,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,0.184458,2.547945,5.817352
4,37.85,-122.25,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,0.172096,2.181467,6.281853


## Question 1

* What is the most frequent observation (mode) for the column ocean_proximity?

In [16]:
ocean_proximity_mode = df.ocean_proximity.mode()[0]
ocean_proximity_mode

'<1H OCEAN'

## Split the data

* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
* Make sure that the target value (median_house_value) is not in your dataframe.

In [17]:
from sklearn.model_selection import train_test_split 

In [18]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state=42)

In [19]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state=42)

In [20]:
len(df_train), len(df_val), len(df_test)

(12384, 4128, 4128)

In [21]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [22]:
y_train = df_train.median_house_value
y_val = df_val.median_house_value
y_test = df_test.median_house_value

In [23]:
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [24]:
df_train.head()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,bedrooms_per_room,population_per_household,rooms_per_household
0,34.43,-119.67,39.0,1467.0,381.0,1404.0,374.0,2.3681,<1H OCEAN,0.259714,3.754011,3.922460
1,33.74,-118.32,24.0,6097.0,794.0,2248.0,806.0,10.1357,NEAR OCEAN,0.130228,2.789082,7.564516
2,39.13,-121.62,41.0,1317.0,309.0,856.0,337.0,1.6719,INLAND,0.234624,2.540059,3.908012
3,34.24,-118.63,9.0,4759.0,924.0,1884.0,915.0,4.8333,<1H OCEAN,0.194158,2.059016,5.201093
4,37.52,-122.30,38.0,2769.0,387.0,994.0,395.0,5.5902,NEAR OCEAN,0.139762,2.516456,7.010127


## Question 2

* Create the correlation matrix for the numerical features of your train dataset.
* In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
* What are the two features that have the biggest correlation in this dataset?

In [26]:
df.dtypes

latitude                    float64
longitude                   float64
housing_median_age          float64
total_rooms                 float64
total_bedrooms              float64
population                  float64
households                  float64
median_income               float64
median_house_value          float64
ocean_proximity              object
bedrooms_per_room           float64
population_per_household    float64
rooms_per_household         float64
dtype: object

In [27]:
numerical = ['latitude', 'longitude', 'housing_median_age',
            'total_rooms', 'total_bedrooms', 'population',
            'households', 'median_income', 'bedrooms_per_room',
             'population_per_household', 'rooms_per_household']

In [28]:
num_numerical = len(numerical)
corr_matrix = np.zeros((num_numerical, num_numerical))
for i in range(num_numerical):
    for j in range(num_numerical):
        corr_matrix[i, j] = df_train[numerical[i]].corr(df_train[numerical[j]])

In [29]:
position = abs(corr_matrix - np.eye(num_numerical)).argmax()

In [30]:
numerical[position // num_numerical], numerical[position % num_numerical]

('total_bedrooms', 'households')

In [32]:
# another approach 
df_train[numerical].corr().round(3)

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,bedrooms_per_room,population_per_household,rooms_per_household
latitude,1.000,-0.925,0.002,-0.026,-0.060,-0.100,-0.064,-0.077,-0.125,-0.002,0.119
longitude,-0.925,1.000,-0.100,0.036,0.064,0.092,0.050,-0.016,0.102,0.011,-0.035
housing_median_age,0.002,-0.100,1.000,-0.364,-0.324,-0.292,-0.306,-0.120,0.129,0.012,-0.181
total_rooms,-0.026,0.036,-0.364,1.000,0.932,0.853,0.921,0.199,-0.194,-0.029,0.169
total_bedrooms,-0.060,0.064,-0.324,0.932,1.000,0.877,0.979,-0.010,0.078,-0.034,0.010
population,-0.100,0.092,-0.292,0.853,0.877,1.000,0.907,-0.001,0.032,0.065,-0.076
households,-0.064,0.050,-0.306,0.921,0.979,0.907,1.000,0.012,0.058,-0.033,-0.086
median_income,-0.077,-0.016,-0.120,0.199,-0.010,-0.001,0.012,1.000,-0.617,-0.000,0.394
bedrooms_per_room,-0.125,0.102,0.129,-0.194,0.078,0.032,0.058,-0.617,1.000,-0.003,-0.501
population_per_household,-0.002,0.011,0.012,-0.029,-0.034,0.065,-0.033,-0.000,-0.003,1.000,0.002


In [33]:
corr_matr = df_train[numerical].corr().abs().unstack()
corr_matr[corr_matr == 1] = 0
corr_matr.sort_values(ascending = False).drop_duplicates().head(1)

total_bedrooms  households    0.979399
dtype: float64

## Make median_house_value binary
* We need to turn the median_house_value variable from numeric into binary.
* Let's create a variable above_average which is 1 if the median_house_value is above its mean value and 0 otherwise.

In [34]:
mean_med_val = df.median_house_value.mean()
above_average_train = (y_train > mean_med_val).astype(int)
above_average_val = (y_val > mean_med_val).astype(int)
above_average_test = (y_test > mean_med_val).astype(int)

In [35]:
mean_med_val

206855.81690891474

In [36]:
y_train.mean()

206807.7419250646

## Question 3

* Calculate the mutual information score with the (binarized) price for the categorical variable that we have. Use the training set only.
* What is the value of mutual information?
* Round it to 2 decimal digits using round(score, 2)

In [37]:
from sklearn.metrics import mutual_info_score

In [38]:
categorical = ['ocean_proximity']

In [39]:
score = mutual_info_score(df_train[categorical[0]], above_average_train)
round(score, 2)

0.1

## Question 4

* Now let's train a logistic regression
* Remember that we have one categorical variable ocean_proximity in the data. Include it using one-hot encoding.
* Fit the model on the training dataset.
* To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [41]:
from sklearn.feature_extraction import DictVectorizer

In [42]:
train_dicts = df_train[numerical + categorical].to_dict(orient='records')

In [43]:
train_dicts[0]

{'latitude': 34.43,
 'longitude': -119.67,
 'housing_median_age': 39.0,
 'total_rooms': 1467.0,
 'total_bedrooms': 381.0,
 'population': 1404.0,
 'households': 374.0,
 'median_income': 2.3681,
 'bedrooms_per_room': 0.25971370143149286,
 'population_per_household': 3.7540106951871657,
 'rooms_per_household': 3.9224598930481283,
 'ocean_proximity': '<1H OCEAN'}

In [44]:
dv = DictVectorizer(sparse=False)

In [45]:
X_train = dv.fit_transform(train_dicts)

In [46]:
X_train

array([[2.59713701e-01, 3.74000000e+02, 3.90000000e+01, ...,
        3.92245989e+00, 3.81000000e+02, 1.46700000e+03],
       [1.30227981e-01, 8.06000000e+02, 2.40000000e+01, ...,
        7.56451613e+00, 7.94000000e+02, 6.09700000e+03],
       [2.34624146e-01, 3.37000000e+02, 4.10000000e+01, ...,
        3.90801187e+00, 3.09000000e+02, 1.31700000e+03],
       ...,
       [1.82879377e-01, 6.02000000e+02, 1.80000000e+01, ...,
        5.54983389e+00, 6.11000000e+02, 3.34100000e+03],
       [2.29126214e-01, 3.50000000e+02, 1.60000000e+01, ...,
        4.41428571e+00, 3.54000000e+02, 1.54500000e+03],
       [2.09574468e-01, 2.15000000e+02, 3.50000000e+01, ...,
        4.37209302e+00, 1.97000000e+02, 9.40000000e+02]])

In [47]:
dv.get_feature_names_out()

array(['bedrooms_per_room', 'households', 'housing_median_age',
       'latitude', 'longitude', 'median_income',
       'ocean_proximity=<1H OCEAN', 'ocean_proximity=INLAND',
       'ocean_proximity=ISLAND', 'ocean_proximity=NEAR BAY',
       'ocean_proximity=NEAR OCEAN', 'population',
       'population_per_household', 'rooms_per_household',
       'total_bedrooms', 'total_rooms'], dtype=object)

In [48]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)

In [49]:
model.fit(X_train, above_average_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [50]:
model.classes_

array([0, 1])

In [51]:
model.coef_[0].round(3)

array([ 0.113,  0.004,  0.036,  0.103,  0.083,  1.196,  0.452, -1.599,
        0.012,  0.309,  0.763, -0.002,  0.01 , -0.014,  0.002, -0.   ])

In [52]:
model.intercept_[0]

-0.06226206219069978

Validation of the model

In [53]:
val_dicts = df_val[numerical + categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [54]:
y_pred = model.predict(X_val)

In [55]:
original_accuracy = (above_average_val == y_pred).mean()
round(original_accuracy, 2)

0.84

## Question 5

* Let's find the least useful feature using the feature elimination technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
* Which of following feature has the smallest difference?

total_rooms
total_bedrooms
population
households


In [63]:
features_list = list(dv.get_feature_names_out())
features_list 

['bedrooms_per_room',
 'households',
 'housing_median_age',
 'latitude',
 'longitude',
 'median_income',
 'ocean_proximity=<1H OCEAN',
 'ocean_proximity=INLAND',
 'ocean_proximity=ISLAND',
 'ocean_proximity=NEAR BAY',
 'ocean_proximity=NEAR OCEAN',
 'population',
 'population_per_household',
 'rooms_per_household',
 'total_bedrooms',
 'total_rooms']

In [58]:
accuracy_score = np.zeros(len(features_list))
for i in range(len(features_list)):
    model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    X_train_part = np.delete(X_train, i, 1)
    X_val_part = np.delete(X_val, i, 1)
    model.fit(X_train_part, above_average_train)
    y_pred = model.predict(X_val_part)
    acc_score = (above_average_val == y_pred).mean()
    accuracy_score[i] = acc_score

In [59]:
accuracy_score

array([0.83624031, 0.83406008, 0.8316376 , 0.83236434, 0.83187984,
       0.78536822, 0.83478682, 0.83575581, 0.83575581, 0.83624031,
       0.83575581, 0.82630814, 0.83575581, 0.83527132, 0.8372093 ,
       0.83624031])

In [60]:
accuracy_difference = abs(accuracy_score - original_accuracy)
accuracy_difference

array([0.00024225, 0.00193798, 0.00436047, 0.00363372, 0.00411822,
       0.05062984, 0.00121124, 0.00024225, 0.00024225, 0.00024225,
       0.00024225, 0.00968992, 0.00024225, 0.00072674, 0.00121124,
       0.00024225])

In [61]:
list_of_diffs = np.array(list(zip(features_list, accuracy_difference)))

In [64]:
chosen = ['total_rooms', 'total_bedrooms', 'population', 'households']
ind = []
for i in chosen:
    ind.append(features_list.index(i))

In [65]:
chosen_diffs = list_of_diffs[ind]

In [66]:
chosen_diffs[chosen_diffs[:, 1].argsort()]

array([['total_rooms', '0.00024224806201555982'],
       ['total_bedrooms', '0.001211240310077577'],
       ['households', '0.0019379844961240345'],
       ['population', '0.009689922480620172']], dtype='<U32')

## Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column 'median_house_value'. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model (model = Ridge(alpha=a, solver="sag", random_state=42)) on the training data.
* This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

In [67]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [68]:
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)
y_test_log = np.log1p(y_test)

In [69]:
alphas = [0, 0.01, 0.1, 1, 10]
rmse = []
for alpha in alphas:
    model = Ridge(alpha=alpha, solver="sag", random_state=42)
    model.fit(X_train, y_train_log)
    y_pred = model.predict(X_val)
    rmse.append( mean_squared_error(y_val, y_pred))

In [70]:
rmse

[57138138586.65307,
 57138138586.65307,
 57138138586.653145,
 57138138586.653885,
 57138138586.66134]

As we can see that the least rmse is when alpha = 0 or alpha = 0.01

In [71]:
alphas[np.array(rmse).argmin()]

0